In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tadgan import tadgan, telemanom
%matplotlib inline

# preprocess

In [ ]:
train_csv = 'processed_train_T1.csv'
src_file = 'telemanom/data/test/T-1.npy'

In [ ]:
label_df = pd.read_csv('telemanom/labeled_anomalies.csv')

In [ ]:
label_df

In [ ]:
anomaly_sequences = eval(label_df[label_df['chan_id'] == 'T-1']['anomaly_sequences'].item())

In [ ]:
train_size = sorted(anomaly_sequences, key=lambda x: x[0])[0][0]

In [ ]:
telemanom.process(src_file, train_csv, train_range=train_size)

# train

In [ ]:
train_dataset = telemanom.TelemanomDataset(train_csv)
tadgan.set_seed()
gan = tadgan.TadGAN(train_dataset)

In [ ]:
(cx_loss, cz_loss, g_loss) = gan.train(num_epoch=100,debug=True)

In [ ]:
values = cx_loss
title = 'cx_loss'
start = 0
end = values.shape[0]

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end, end - start)
ax.plot(x,values[start:end])
plt.show()

In [ ]:
values = g_loss
title = 'g_loss'
start = 0
end = values.shape[0]

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end, end - start)
ax.plot(x,values[start:end])
plt.show()

In [ ]:
raw = gan.raw
reconstructed = gan.reconstruct(raw)

In [ ]:
start = 0
end = raw.shape[0]
title = 'train'

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
x = np.linspace(start, end, end - start)
ax.set_title(title)
for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(raw, 'r', label='raw')
ax.plot(reconstructed, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(-1.1, 1.1)
ax.legend()
plt.show()

# test

In [ ]:
left_edge = int(train_size*0.8)
test_values =np.load(src_file)[left_edge:-1,0].reshape(-1, 1)
test_reconstructed = gan.reconstruct(test_values)

In [ ]:
title = 'test'
start = left_edge
end = left_edge + test_values.shape[0]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end - 1, test_values.shape[0])

for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(x, test_values, 'r', label='raw')
ax.plot(x, test_reconstructed, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(-1.1, 1.1)
ax.legend()
plt.show()

In [ ]:
anomaly_score1 = gan.anomaly_score(test_values, test_reconstructed, distance='point', alpha=1)

In [ ]:
values = anomaly_score1
title = 'distance=point'
start = left_edge
end = left_edge + test_values.shape[0]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end - 1, test_values.shape[0])
for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(x, values, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(bottom=0)
ax.legend()
plt.show()

In [ ]:
anomaly_score2 = gan.anomaly_score(test_values, test_reconstructed, distance='dtw', combination=None, score_window=10)

In [ ]:
values = anomaly_score2
title = 'distance=dtw, score_window=10'
start = left_edge
end = left_edge + test_values.shape[0]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end - 1, test_values.shape[0])
for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(x, values, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(bottom=0)
ax.legend()
plt.show()

In [ ]:
anomaly_score3 = gan.anomaly_score(test_values, test_reconstructed, distance='point', alpha=0)

In [ ]:
values = anomaly_score3
title = 'alpha=0'
start = left_edge
end = left_edge + test_values.shape[0]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end - 1, test_values.shape[0])
for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(x, values, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(bottom=0)
ax.legend()
plt.show()

In [ ]:
anomaly_score4 = gan.anomaly_score(test_values, test_reconstructed, distance='dtw', alpha=0.5)

In [ ]:
values = anomaly_score4
title = 'distance=dtw, score_window=10, combination=add, alpha=0.50'
start = left_edge
end = left_edge + test_values.shape[0]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.set_title(title)
x = np.linspace(start, end - 1, test_values.shape[0])
for [anomaly_start, anomaly_end] in anomaly_sequences:
    ax.axvspan(anomaly_start, anomaly_end, color='gray', alpha=0.25)
ax.plot(x, values, 'b', label='reconstructed')
plt.xlim(start, end)
plt.ylim(bottom=0)
ax.legend()
plt.show()